In [1]:
import ipywidgets as widgets

path = widgets.Text(
    placeholder='Type something',
    description='File path:',
    disabled=False
)

display(path)


Text(value='', description='File path:', placeholder='Type something')

In [4]:
%matplotlib notebook
import ipyvolume as ipv

from skimage import data, io, filters
## C:\Users\inst\SynologyDrive\Master\Exp\Hi-am\data\2019-04-10 - Issa - Fe3Si -S08 (unlabelled, green)_2um scan_recon.tiff
stack = io.imread(path.value)[:,:,0:50]

In [5]:
import numpy.ma as ma

ma_stack = ma.masked_where(stack <= 0, stack)

In [ ]:
from skimage.restoration import denoise_nl_means
import ipyvolume as ipv

cleaned = denoise_nl_means(ma_stack,patch_size=3,multichannel=False)
ipv.quickvolshow(cleaned[:,:,20])

C:\Users\inst\.conda\envs\machine_learning\lib\site-packages\skimage\restoration\non_local_means.py:132: UserWarning: denoise_nl_means will default to multichannel=False in v0.15
  warn('denoise_nl_means will default to multichannel=False in v0.15')


In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2,n_jobs=2)

bined = kmeans.predict(cleaned)

ipv.quickvolshow(bined[:,:,20])